In [1]:
import pandas as pd
from deviation_functions import normalise, calc_dev_scores, score_ds_q123
import numpy as np

In [2]:
ri_data_path = "../data/reference_data/"
mdd_data_path = "../data/depression_data/"


# where you want to store the prediction data
location_to_save_data = "../data/prediction_data/"

In [3]:
pd.read_csv(ri_data_path+"reference_descriptives.csv", index_col=0)

,dataset,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
2,clac_males,"(3.686937999154304, 0.49415676542521314)","(4.3263330412651735, 0.4461043928532965)","(0.8518480106705816, 0.06863502555077977)","(0.24725776991129317, 0.0940298671910015)","(0.5927437801552339, 0.20629731695112002)","(117.81970727376503, 29.354948559032835)","(3.117727471894877, 1.38293729213957)","(71.83124907753657, 4.869715529612232)","(1.6094610277037598, 126.38332346541796)","(5.357098162890241, 2.147612916150901)","(-16.188623839509784, 4.83573795115372)","(-0.006795774336491557, 0.0018013356878964013)","(8.648310687116682, 1.705465684045437)","(481.08365873890375, 71.61009609075668)","(201.03667683428932, 65.16900811762754)","(168.7434337011567, 68.98843235778135)","(200.99299430959064, 68.71150803959135)","(1510.1585360480178, 104.34301160030348)","(426.8070876071593, 61.442693673022255)","(313.1959398886768, 92.84191248771229)","(409.58946847774286, 109.1338594022463)","(531.459005482349, 233.80467598334266)","(398.2230345999431, 147.61321435854677)","(4.6553844239925715, 1.613400771901009)","(73.65625248231085, 59.041679437498175)"
3,clac_females,"(3.8276591254940353, 0.5265453411037165)","(4.394874192454035, 0.43751427635886153)","(0.869766928973627, 0.07332294578815503)","(0.2263646563662828, 0.10638196912567187)","(0.5508401458511046, 0.16398355621075136)","(195.7247923658587, 27.943870668467966)","(3.4923637662152753, 0.9728680275805698)","(73.44376443332233, 4.334449618285865)","(6.693620524779084, 123.31132404447959)","(9.439438442630854, 2.376319381688551)","(-15.87947760219253, 4.6561875899956116)","(-0.006158778927092862, 0.0017534436661733976)","(9.43607437610024, 1.5597188027063704)","(529.0011814430956, 63.95079988949581)","(203.8570423045786, 48.869218033388925)","(192.5716438740452, 53.44909453296275)","(268.00814987790824, 65.78456521172112)","(1607.980525188241, 141.30641568052218)","(501.589605647992, 85.41419432721061)","(446.5019042102586, 107.14425920907519)","(602.0584694531924, 130.21775516593894)","(566.1186377158197, 200.86348848189147)","(395.542366651839, 135.06204333339437)","(5.120149074391844, 1.6480290318929234)","(99.63288161368564, 75.31693788140147)"
4,common_males,"(3.969768703489275, 0.901547763188605)","(4.14650563441203, 0.8450807405830122)","(0.9568429018275012, 0.09173133500449708)","(0.08469267301183372, 0.1655812330911897)","(0.13132122274826113, 0.3455700334194028)","(128.51757458221658, 32.05163820145666)","(3.153468031224966, 1.4284810493021933)","(66.69940961690456, 8.178476484494288)","(-3.3333156733543454, 506.3943141788426)","(5.952800804167977, 2.53707792079255)","(-17.42129886583369, 4.974427981655563)","(-0.00642580361997799, 0.0018862228854504288)","(8.15561221692547, 1.5911861525931466)","(472.32366313176055, 79.62660301894766)","(193.5970062978888, 87.75800918174676)","(194.37190973253286, 82.85549366617997)","(205.79217118293846, 89.27391886604313)","(1535.6135731288005, 160.31314446562854)","(391.0939583411243, 88.40055787766869)","(363.921357774823, 121.96755333980647)","(416.1733777710544, 153.25615811493137)","(461.0397125382718, 174.63055576708902)","(382.61983359243504, 151.15198827316894)","(5.341484963548701, 1.8855932204272083)","(86.37788670064607, 71.66176034186762)"
5,common_females,"(4.056499926287583, 0.8767711418891906)","(4.242115495493237, 0.8264258916605276)","(0.9564765209233418, 0.08845690867537337)","(0.08634382096004158, 0.16189914620227272)","(0.13297826564215148, 0.2507070053873603)","(204.3200383725392, 36.61263439862343)","(3.3360476657417286, 1.2471963005052253)","(65.85694812509153, 9.130588027860236)","(1.8508750115616577, 110.27091121232054)","(9.655722725941352, 2.

### Load data

In [4]:
# ri datasets
clac_female_reference_intervals = pd.read_csv(ri_data_path + "clac_female_reference_intervals.csv", index_col=0)
clac_male_reference_intervals = pd.read_csv(ri_data_path + "clac_male_reference_intervals.csv",index_col=0)

common_female_reference_intervals = pd.read_csv(ri_data_path + "common_female_reference_intervals.csv",index_col=0)
common_male_reference_intervals = pd.read_csv(ri_data_path + "common_male_reference_intervals.csv",index_col=0)

In [5]:
## checking the reference interval dfs
clac_male_reference_intervals.head()

,feature,RI_lower_limit,RI_upper_limit,CI_for_lower_limit,CI_for_upper_limit,ci_lower_width/ri_width,ci_upper_width/ri_width,valid_CI_lower_limit,valid_CI_upper_limit,median,q1,q3
0,Speaking_Rate,-1.883544,1.862419,"(-2.103771545725876, -1.6563509045876526)","(1.715459199614647, 2.2428457041148167)",0.119441,0.140788,True,True,-0.025285,-0.645892,0.722881
1,Articulation_Rate,-1.820120,2.151624,"(-2.0790442854931874, -1.5476706161395857)","(1.8807474517867075, 2.3347471494671304)",0.133789,0.114307,True,True,0.006440,-0.650212,0.620862
2,Phonation_Ratio,-2.410274,1.592982,"(-2.518333166338251, -2.08599944367207)","(1.4050946767415433, 2.160929076097077)",0.107996,0.188805,True,True,0.071917,-0.494887,0.681912
3,Pause_Rate,-1.754677,2.143551,"(-2.632467090556839, -1.6750344803969097)","(1.8802912395459805, 2.4921166510584367)",0.245607,0.156950,False,True,-0.038674,-0.674937,0.529467
4,Mean_Pause_Duration,-1.116823,1.992023,"(-2.8764196783796816, -1.0647374936862448)","(1.7045481314880082, 2.7915574423040717)",0.582751,0.349650,False,False,-0.090958,-0.528997,0.384608


Load in mdd data and split into male and female for normalisation

In [6]:
mdd = pd.read_csv(mdd_data_path + "synthetic_mdd_data.csv", index_col=0)

# 0 = males, 1 = females
mdd_males = mdd.loc[mdd["gender"] == 0,:]
mdd_females = mdd.loc[mdd["gender"] == 1,:]

mdd descriptives

In [7]:
features_for_descriptives = ["speaking_rate", "mean_f0", "cepstral_peak_prominence", "mean_f1_loc", "mean_f2_loc", "spectral_gravity"]
mdd_males[features_for_descriptives].describe().map(lambda x: f"{x:0.2f}")

,speaking_rate,mean_f0,cepstral_peak_prominence,mean_f1_loc,mean_f2_loc,spectral_gravity
count,494.00,494.00,494.00,494.00,494.00,494.00
mean,4.08,188.42,0.82,598.90,1657.15,2738.54
std,1.16,62.72,0.36,175.74,482.69,1275.89
min,2.02,80.58,0.20,302.71,802.66,503.78
25%,3.03,135.61,0.53,444.70,1268.65,1651.52
50%,4.20,184.09,0.79,608.95,1632.82,2764.90
75%,5.04,244.43,1.12,756.57,2050.41,3788.50
max,5.98,299.80,1.49,899.24,2497.94,4996.15


In [8]:
mdd_females[features_for_descriptives].describe().map(lambda x: f"{x:0.2f}")

,speaking_rate,mean_f0,cepstral_peak_prominence,mean_f1_loc,mean_f2_loc,spectral_gravity
count,506.00,506.00,506.00,506.00,506.00,506.00
mean,4.01,188.69,0.89,603.12,1645.97,2758.30
std,1.17,65.33,0.36,170.30,507.06,1310.94
min,2.01,80.21,0.21,302.50,804.87,507.24
25%,3.02,128.31,0.60,462.74,1216.25,1581.22
50%,4.01,188.02,0.93,605.72,1630.90,2695.26
75%,5.05,247.95,1.21,748.22,2113.57,3883.72
max,6.00,299.81,1.50,899.13,2496.10,4990.50


### Normalising mdd Data

ref_descriptives contains tuples of the mean and sd for each speech feature for each of the three reference interval datasets, separated by male and female. We need this to normalise the mdd data using reference data.

In [9]:
ref_descriptives = pd.read_csv(ri_data_path + "reference_descriptives.csv", index_col=0)

In [10]:
ref_descriptives

,dataset,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
2,clac_males,"(3.686937999154304, 0.49415676542521314)","(4.3263330412651735, 0.4461043928532965)","(0.8518480106705816, 0.06863502555077977)","(0.24725776991129317, 0.0940298671910015)","(0.5927437801552339, 0.20629731695112002)","(117.81970727376503, 29.354948559032835)","(3.117727471894877, 1.38293729213957)","(71.83124907753657, 4.869715529612232)","(1.6094610277037598, 126.38332346541796)","(5.357098162890241, 2.147612916150901)","(-16.188623839509784, 4.83573795115372)","(-0.006795774336491557, 0.0018013356878964013)","(8.648310687116682, 1.705465684045437)","(481.08365873890375, 71.61009609075668)","(201.03667683428932, 65.16900811762754)","(168.7434337011567, 68.98843235778135)","(200.99299430959064, 68.71150803959135)","(1510.1585360480178, 104.34301160030348)","(426.8070876071593, 61.442693673022255)","(313.1959398886768, 92.84191248771229)","(409.58946847774286, 109.1338594022463)","(531.459005482349, 233.80467598334266)","(398.2230345999431, 147.61321435854677)","(4.6553844239925715, 1.613400771901009)","(73.65625248231085, 59.041679437498175)"
3,clac_females,"(3.8276591254940353, 0.5265453411037165)","(4.394874192454035, 0.43751427635886153)","(0.869766928973627, 0.07332294578815503)","(0.2263646563662828, 0.10638196912567187)","(0.5508401458511046, 0.16398355621075136)","(195.7247923658587, 27.943870668467966)","(3.4923637662152753, 0.9728680275805698)","(73.44376443332233, 4.334449618285865)","(6.693620524779084, 123.31132404447959)","(9.439438442630854, 2.376319381688551)","(-15.87947760219253, 4.6561875899956116)","(-0.006158778927092862, 0.0017534436661733976)","(9.43607437610024, 1.5597188027063704)","(529.0011814430956, 63.95079988949581)","(203.8570423045786, 48.869218033388925)","(192.5716438740452, 53.44909453296275)","(268.00814987790824, 65.78456521172112)","(1607.980525188241, 141.30641568052218)","(501.589605647992, 85.41419432721061)","(446.5019042102586, 107.14425920907519)","(602.0584694531924, 130.21775516593894)","(566.1186377158197, 200.86348848189147)","(395.542366651839, 135.06204333339437)","(5.120149074391844, 1.6480290318929234)","(99.63288161368564, 75.31693788140147)"
4,common_males,"(3.969768703489275, 0.901547763188605)","(4.14650563441203, 0.8450807405830122)","(0.9568429018275012, 0.09173133500449708)","(0.08469267301183372, 0.1655812330911897)","(0.13132122274826113, 0.3455700334194028)","(128.51757458221658, 32.05163820145666)","(3.153468031224966, 1.4284810493021933)","(66.69940961690456, 8.178476484494288)","(-3.3333156733543454, 506.3943141788426)","(5.952800804167977, 2.53707792079255)","(-17.42129886583369, 4.974427981655563)","(-0.00642580361997799, 0.0018862228854504288)","(8.15561221692547, 1.5911861525931466)","(472.32366313176055, 79.62660301894766)","(193.5970062978888, 87.75800918174676)","(194.37190973253286, 82.85549366617997)","(205.79217118293846, 89.27391886604313)","(1535.6135731288005, 160.31314446562854)","(391.0939583411243, 88.40055787766869)","(363.921357774823, 121.96755333980647)","(416.1733777710544, 153.25615811493137)","(461.0397125382718, 174.63055576708902)","(382.61983359243504, 151.15198827316894)","(5.341484963548701, 1.8855932204272083)","(86.37788670064607, 71.66176034186762)"
5,common_females,"(4.056499926287583, 0.8767711418891906)","(4.242115495493237, 0.8264258916605276)","(0.9564765209233418, 0.08845690867537337)","(0.08634382096004158, 0.16189914620227272)","(0.13297826564215148, 0.2507070053873603)","(204.3200383725392, 36.61263439862343)","(3.3360476657417286, 1.2471963005052253)","(65.85694812509153, 9.130588027860236)","(1.8508750115616577, 110.27091121232054)","(9.655722725941352, 2.

The ref_descriptives dataset was exported with the columns as strings, so they need to be converted back to tuples.

In [11]:
# convert strings to tuples
import ast

for col in ref_descriptives.columns[1:]:
    ref_descriptives[col] = ref_descriptives[col].apply(ast.literal_eval)

# columns to lowercase for compatibility with mdd
ref_descriptives.columns = map(str.lower, ref_descriptives.columns)

In [12]:
# should all be tuples
for col in ref_descriptives.columns[1:]:
    print(col)
    for i in ref_descriptives[col]:
        print(type(i))

speaking_rate
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
articulation_rate
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
phonation_ratio
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
pause_rate
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
mean_pause_duration
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
mean_f0
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
stdev_f0_semitone
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
mean_db
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
range_ratio_db
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
hnr_db
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
spectral_slope
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
spectral_tilt
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
cepstral_peak_prominence
<class 'tuple'>
<class 'tuple'>
<class 'tup

The normalise function will use the mean and sd from each feature column of the reference dataset to normalise the mdd data. 

In [13]:
speech_features = [
    'speaking_rate', 'articulation_rate', 'phonation_ratio', 'pause_rate', 'mean_pause_duration', 'mean_f0', 'stdev_f0_semitone', 'mean_db', 'range_ratio_db', 'hnr_db', 'spectral_slope', 
    'spectral_tilt', 'cepstral_peak_prominence', 'mean_f1_loc', 'std_f1_loc', 'mean_b1_loc', 'std_b1_loc', 'mean_f2_loc', 'std_f2_loc', 'mean_b2_loc', 'std_b2_loc', 'spectral_gravity', 
    'spectral_std_dev', 'spectral_skewness', 'spectral_kurtosis'
    ]

It is a wise idea to check that the raw mean/sd are somewhat similar before normalisation. 

In [14]:
mdd_datasets = [(mdd_males, "mdd_males"), (mdd_females, "mdd_females")]
descriptives_comparison = ref_descriptives.copy()

# want to add a row with the (mean, sd) of each feature for each of the 6 dfs.
for df in mdd_datasets:
    data = df[0]
    data_name = df[1]
    
    data_row = {"dataset": data_name}
    
    for feature in speech_features:
        data_mean = data[feature].mean()
        data_std = data[feature].std()
        
        data_row[feature] = (data_mean, data_std)
    
    # print(data_row)
    descriptives_comparison = pd.concat([descriptives_comparison, pd.DataFrame([data_row])])

descriptives_comparison

,dataset,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
2,clac_males,"(3.686937999154304, 0.49415676542521314)","(4.3263330412651735, 0.4461043928532965)","(0.8518480106705816, 0.06863502555077977)","(0.24725776991129317, 0.0940298671910015)","(0.5927437801552339, 0.20629731695112002)","(117.81970727376503, 29.354948559032835)","(3.117727471894877, 1.38293729213957)","(71.83124907753657, 4.869715529612232)","(1.6094610277037598, 126.38332346541796)","(5.357098162890241, 2.147612916150901)","(-16.188623839509784, 4.83573795115372)","(-0.006795774336491557, 0.0018013356878964013)","(8.648310687116682, 1.705465684045437)","(481.08365873890375, 71.61009609075668)","(201.03667683428932, 65.16900811762754)","(168.7434337011567, 68.98843235778135)","(200.99299430959064, 68.71150803959135)","(1510.1585360480178, 104.34301160030348)","(426.8070876071593, 61.442693673022255)","(313.1959398886768, 92.84191248771229)","(409.58946847774286, 109.1338594022463)","(531.459005482349, 233.80467598334266)","(398.2230345999431, 147.61321435854677)","(4.6553844239925715, 1.613400771901009)","(73.65625248231085, 59.041679437498175)"
3,clac_females,"(3.8276591254940353, 0.5265453411037165)","(4.394874192454035, 0.43751427635886153)","(0.869766928973627, 0.07332294578815503)","(0.2263646563662828, 0.10638196912567187)","(0.5508401458511046, 0.16398355621075136)","(195.7247923658587, 27.943870668467966)","(3.4923637662152753, 0.9728680275805698)","(73.44376443332233, 4.334449618285865)","(6.693620524779084, 123.31132404447959)","(9.439438442630854, 2.376319381688551)","(-15.87947760219253, 4.6561875899956116)","(-0.006158778927092862, 0.0017534436661733976)","(9.43607437610024, 1.5597188027063704)","(529.0011814430956, 63.95079988949581)","(203.8570423045786, 48.869218033388925)","(192.5716438740452, 53.44909453296275)","(268.00814987790824, 65.78456521172112)","(1607.980525188241, 141.30641568052218)","(501.589605647992, 85.41419432721061)","(446.5019042102586, 107.14425920907519)","(602.0584694531924, 130.21775516593894)","(566.1186377158197, 200.86348848189147)","(395.542366651839, 135.06204333339437)","(5.120149074391844, 1.6480290318929234)","(99.63288161368564, 75.31693788140147)"
4,common_males,"(3.969768703489275, 0.901547763188605)","(4.14650563441203, 0.8450807405830122)","(0.9568429018275012, 0.09173133500449708)","(0.08469267301183372, 0.1655812330911897)","(0.13132122274826113, 0.3455700334194028)","(128.51757458221658, 32.05163820145666)","(3.153468031224966, 1.4284810493021933)","(66.69940961690456, 8.178476484494288)","(-3.3333156733543454, 506.3943141788426)","(5.952800804167977, 2.53707792079255)","(-17.42129886583369, 4.974427981655563)","(-0.00642580361997799, 0.0018862228854504288)","(8.15561221692547, 1.5911861525931466)","(472.32366313176055, 79.62660301894766)","(193.5970062978888, 87.75800918174676)","(194.37190973253286, 82.85549366617997)","(205.79217118293846, 89.27391886604313)","(1535.6135731288005, 160.31314446562854)","(391.0939583411243, 88.40055787766869)","(363.921357774823, 121.96755333980647)","(416.1733777710544, 153.25615811493137)","(461.0397125382718, 174.63055576708902)","(382.61983359243504, 151.15198827316894)","(5.341484963548701, 1.8855932204272083)","(86.37788670064607, 71.66176034186762)"
5,common_females,"(4.056499926287583, 0.8767711418891906)","(4.242115495493237, 0.8264258916605276)","(0.9564765209233418, 0.08845690867537337)","(0.08634382096004158, 0.16189914620227272)","(0.13297826564215148, 0.2507070053873603)","(204.3200383725392, 36.61263439862343)","(3.3360476657417286, 1.2471963005052253)","(65.85694812509153, 9.130588027860236)","(1.8508750115616577, 110.27091121232054)","(9.655722725941352, 2.

Now to create the normalised mdd datasets. 

In [15]:
# normalise males and females separately
clac_males_mdd = normalise(mdd_males, speech_features, ref_descriptives, "clac_males")
clac_females_mdd = normalise(mdd_females, speech_features, ref_descriptives, "clac_females")

common_males_mdd = normalise(mdd_males, speech_features, ref_descriptives, "common_males")
common_females_mdd = normalise(mdd_females, speech_features, ref_descriptives, "common_females")

[INFO] Successfully normalised with clac_males
[INFO] Successfully normalised with clac_females
[INFO] Successfully normalised with common_males
[INFO] Successfully normalised with common_females
[INFO] Successfully normalised with clac_males
[INFO] Successfully normalised with clac_females
[INFO] Successfully normalised with common_males
[INFO] Successfully normalised with common_females


### Check that the normalised mdd datasets look as they should

In [16]:
clac_males_mdd.head(3)

,pid,phq8,age,gender,education_years,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
1,2,5,78,0,16,-2.134221,1.682567,0.315861,7.963752,-0.482617,3.850855,-1.405569,-4.192687,0.006269,3.156215,2.867875,-4976.904672,-4.939189,0.497623,-2.284630,0.621044,-2.298841,-0.956601,-3.716269,-0.148469,-2.876533,15.517867,2.481286,-2.491712,-1.166162
2,3,13,75,0,18,0.449228,-0.006604,-2.682650,15.830404,1.293046,0.079520,-1.737352,0.253741,-0.004767,5.384358,2.556093,-4575.621180,-4.515330,-0.746188,-1.754735,-1.906953,-2.736692,-6.769275,-6.467795,-0.939767,-3.446341,4.552249,4.500247,-2.951525,-1.119409
5,6,12,56,0,8,-1.665366,0.901999,-4.972190,18.017905,5.265343,3.693439,-1.375795,1.308120,0.008207,2.355676,3.221983,-1591.704013,-4.682945,1.247678,-1.826531,-0.420981,-2.530381,-0.267967,-4.728227,-1.899650,-3.233103,8.290300,6.061881,-3.289269,-1.183723


In [17]:
clac_females_mdd.head(3)

,pid,phq8,age,gender,education_years,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
0,1,5,77,1,13,-0.825097,-0.600369,-4.146547,12.792051,-0.695845,-2.932920,-1.908327,1.865961,-0.043340,8.230240,2.599513,-1980.226195,-5.303603,5.293791,-3.431990,-2.988028,-3.951913,-1.959557,-3.924956,-2.171849,-4.223690,10.293154,3.396739,-3.328585,-1.216497
3,4,13,64,1,11,2.313027,5.730632,0.523023,5.430757,4.496305,3.424762,-1.681721,-1.080341,-0.032310,5.211294,2.744255,-3839.758606,-5.543319,1.425187,-3.131451,-1.353645,-3.314927,4.715934,-4.073027,-2.903211,-4.451862,8.008382,5.288009,-3.335410,-1.304416
4,5,20,44,1,7,2.092048,-1.774255,-1.299160,10.446430,1.845954,3.459075,-1.863921,-3.453093,-0.044979,7.493139,3.099370,-2023.704547,-5.239897,-3.153013,-3.275271,-0.733369,-3.634093,1.884413,-5.474802,-3.574497,-4.327079,7.017940,-0.919626,-3.211450,-1.288074


In [18]:
common_males_mdd.head(3)

,pid,phq8,age,gender,education_years,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
1,2,5,78,0,16,-1.483527,1.100993,-0.908258,5.504220,1.047139,3.193091,-1.385776,-1.868973,0.011325,2.436908,3.035719,-4753.121194,-4.984281,0.557538,-1.611789,0.207788,-1.823108,-0.781406,-2.178996,-0.528907,-2.091342,21.179394,2.526423,-2.495889,-1.138317
2,3,13,75,0,18,-0.067486,0.209307,-3.151798,9.971516,2.107169,-0.260941,-1.706981,0.778566,0.008571,4.323010,2.732630,-4369.896983,-4.529981,-0.561051,-1.218290,-1.897112,-2.160109,-4.564701,-4.091441,-1.131245,-2.497103,6.498040,4.498116,-2.889326,-1.099798
5,6,12,56,0,8,-1.226538,0.688944,-4.864872,11.213748,4.478538,3.048919,-1.356951,1.406376,0.011809,1.759259,3.379955,-1520.267427,-4.709634,1.232080,-1.271605,-0.659839,-2.001317,-0.333195,-2.882356,-1.861909,-2.345256,11.502741,6.023189,-3.178315,-1.152785


In [19]:
common_females_mdd.head(3)

,pid,phq8,age,gender,education_years,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
0,1,5,77,1,13,-0.756517,-0.132996,-4.417367,9.270367,1.211593,-2.473255,-1.363245,1.716728,-0.004549,7.042036,2.957414,-1670.882135,-4.926007,4.689297,-2.402908,-2.584647,-2.683609,-1.391072,-2.637124,-1.777979,-3.040147,12.001667,3.383810,-2.592008,-0.944993
3,4,13,64,1,11,1.128086,3.218670,-0.546707,4.433349,4.607697,2.379120,-1.186483,0.318066,0.007786,4.430051,3.098567,-3239.957008,-5.169690,1.354026,-2.178777,-1.346916,-2.242840,3.522889,-2.758391,-2.299501,-3.215108,9.401927,5.194443,-2.596805,-1.003683
4,5,20,44,1,7,0.995377,-0.754457,-2.057135,7.729087,2.874144,2.405309,-1.328607,-0.808320,-0.006381,6.404298,3.444876,-1707.569202,-4.861248,-2.593013,-2.286032,-0.877175,-2.463690,1.438551,-3.906406,-2.778183,-3.119425,8.274946,-0.748520,-2.509673,-0.992774


In [20]:
speech_features

['speaking_rate',
 'articulation_rate',
 'phonation_ratio',
 'pause_rate',
 'mean_pause_duration',
 'mean_f0',
 'stdev_f0_semitone',
 'mean_db',
 'range_ratio_db',
 'hnr_db',
 'spectral_slope',
 'spectral_tilt',
 'cepstral_peak_prominence',
 'mean_f1_loc',
 'std_f1_loc',
 'mean_b1_loc',
 'std_b1_loc',
 'mean_f2_loc',
 'std_f2_loc',
 'mean_b2_loc',
 'std_b2_loc',
 'spectral_gravity',
 'spectral_std_dev',
 'spectral_skewness',
 'spectral_kurtosis']

### Rounding stuff for the mean (SD) table in results

In [21]:
# round CLAC data
clac_female_ris_rounded = clac_female_reference_intervals.copy()
clac_male_ris_rounded = clac_male_reference_intervals.copy()

clac_female_ris_rounded['RI_lower_limit'] = clac_female_ris_rounded['RI_lower_limit'].apply(lambda x: round(x, 2))
clac_male_ris_rounded['RI_lower_limit'] = clac_male_ris_rounded['RI_lower_limit'].apply(lambda x: round(x, 2))

clac_female_ris_rounded['RI_upper_limit'] = clac_female_ris_rounded['RI_upper_limit'].apply(lambda x: round(x, 2))
clac_male_ris_rounded['RI_upper_limit'] = clac_male_ris_rounded['RI_upper_limit'].apply(lambda x: round(x, 2))

# round common voice data
common_female_ris_rounded = common_female_reference_intervals.copy()
common_male_ris_rounded = common_male_reference_intervals.copy()

common_female_ris_rounded['RI_lower_limit'] = common_female_ris_rounded['RI_lower_limit'].apply(lambda x: round(x, 2))
common_male_ris_rounded['RI_lower_limit'] = common_male_ris_rounded['RI_lower_limit'].apply(lambda x: round(x, 2))

common_female_ris_rounded['RI_upper_limit'] = common_female_ris_rounded['RI_upper_limit'].apply(lambda x: round(x, 2))
common_male_ris_rounded['RI_upper_limit'] = common_male_ris_rounded['RI_upper_limit'].apply(lambda x: round(x, 2))

### Determine the deviation scores!

In [22]:
# ensure all features are in lowercase 
common_male_reference_intervals['feature'] = common_male_reference_intervals['feature'].apply(lambda x: x.lower())
common_female_reference_intervals['feature'] = common_female_reference_intervals['feature'].apply(lambda x: x.lower())

clac_male_reference_intervals['feature'] = clac_male_reference_intervals['feature'].apply(lambda x: x.lower())
clac_female_reference_intervals['feature'] = clac_female_reference_intervals['feature'].apply(lambda x: x.lower())

In [23]:
common_males_mdd_dev = calc_dev_scores(common_males_mdd, common_male_reference_intervals, speech_features)
common_females_mdd_dev = calc_dev_scores(common_females_mdd, common_female_reference_intervals, speech_features)

clac_males_mdd_dev = calc_dev_scores(clac_males_mdd, clac_male_reference_intervals, speech_features)
clac_females_mdd_dev = calc_dev_scores(clac_females_mdd, clac_female_reference_intervals, speech_features)

0.6465245699512934
-0.6363883493743973
0.0087816786637174
0.6449676469915782
-0.6355660899904235
-0.0084604399208208
0.4704640709319927
0.4704640709319927
0.4704640709319927
-0.5144338738173846
-0.5144338738173846
-0.5144338738173846
-0.3717307864738716
-0.3717307864738716
-0.3717307864738716
0.3148260989887118
-0.6062217105881708
-0.204894725974236
0.432188444945517
-0.6543319723156905
-0.1549577516298847
0.7384920296620956
-0.571965055815578
0.1448397973448956
0.0149897176785911
0.007470959283874
0.0097978342756847
0.6169983883906729
-0.6617777863193544
-0.0704693189587689
0.6531476798789615
-0.6807261501739206
-0.0293429779216186
0.6463909426349561
-0.7020752353420815
-0.0302476651995595
0.6468681914442678
-0.7227893876451651
-0.0645552324655897
0.4583341461490083
-0.6281741528492868
-0.1417999144364157
0.521040987543088
-0.7390207858426511
-0.2242709052653404
0.4814817375656692
-0.7025164350393746
-0.1855590489696057
0.5988191819238247
-0.735026543355877
-0.1325721277901962
0.64602

/Users/laurenwhite/IS25/deviation/deviation_functions.py:58: RuntimeWarning: divide by zero encountered in scalar divide
  return -np.abs(q1 - value)/ri_length
/Users/laurenwhite/IS25/deviation/deviation_functions.py:55: RuntimeWarning: divide by zero encountered in scalar divide
  return np.abs(q3 - value)/ri_length


Common voice pause features need to be removed

In [24]:
common_males_mdd_dev.drop(["phonation_ratio", "pause_rate", "mean_pause_duration"], axis=1, inplace=True)
common_females_mdd_dev.drop(["phonation_ratio", "pause_rate", "mean_pause_duration"], axis=1, inplace=True)

### combining all the data

In [25]:
# concatenate males and females
clac_mdd_features = pd.concat([clac_females_mdd_dev, clac_males_mdd_dev])
common_mdd_features = pd.concat([common_females_mdd_dev, common_males_mdd_dev])

In [26]:
clac_mdd_features.to_csv(location_to_save_data+"clac_mdd_features.csv")
common_mdd_features.to_csv(location_to_save_data+"common_mdd_features.csv")

### How many people deviate for each feature?

In [27]:
features_of_interest = ["speaking_rate", "mean_f1_loc", "mean_f2_loc", "mean_f0", "spectral_gravity"]

In [28]:
current_df = clac_mdd_features

for col in features_of_interest:
    count_inrange = (current_df[col] == 0).sum()
    count_negative = (current_df[col] < 0).sum()
    count_positive = (current_df[col] > 0).sum()
    print(f"{col}\nin range: {count_inrange}\nnegative: {count_negative}\npositive: {count_positive}\n\n") 

speaking_rate
in range: 155
negative: 346
positive: 499


mean_f1_loc
in range: 113
negative: 275
positive: 612


mean_f2_loc
in range: 78
negative: 413
positive: 509


mean_f0
in range: 124
negative: 280
positive: 596


spectral_gravity
in range: 19
negative: 0
positive: 981


speaking_rate
in range: 155
negative: 346
positive: 499


mean_f1_loc
in range: 113
negative: 275
positive: 612


mean_f2_loc
in range: 78
negative: 413
positive: 509


mean_f0
in range: 124
negative: 280
positive: 596


spectral_gravity
in range: 19
negative: 0
positive: 981




### Age descriptives

In [29]:
clac_mdd_features["age"].describe()

count    1000.000000
mean       48.890000
std        17.487244
min        18.000000
25%        34.000000
50%        49.000000
75%        64.000000
max        79.000000
Name: age, dtype: float64

In [84]:
common_mdd_features["age"].describe()

count    1000.000000
mean       48.890000
std        17.487244
min        18.000000
25%        34.000000
50%        49.000000
75%        64.000000
max        79.000000
Name: age, dtype: float64

### number of mdd observations etc.

This section only applies if you use repeated measures data.

In [30]:
# grouping variable (participant id for repeated measures)
group_var = "pid"

In [31]:
# Count the number of observations per group
group_counts = clac_mdd_features.groupby(group_var).size()

# Calculate mean and standard deviation of the number of observations per group
mean_count = group_counts.mean()
std_dev_count = group_counts.std()

print(mean_count)
print(std_dev_count)
print(len(clac_mdd_features))

1.0
0.0
1000
1.0
0.0
1000


### Gender distribution in mdd

In [32]:
clac_mdd_features.head()

,pid,phq8,age,gender,education_years,speaking_rate,articulation_rate,phonation_ratio,pause_rate,mean_pause_duration,mean_f0,stdev_f0_semitone,mean_db,range_ratio_db,hnr_db,spectral_slope,spectral_tilt,cepstral_peak_prominence,mean_f1_loc,std_f1_loc,mean_b1_loc,std_b1_loc,mean_f2_loc,std_f2_loc,mean_b2_loc,std_b2_loc,spectral_gravity,spectral_std_dev,spectral_skewness,spectral_kurtosis
0,1,5,77,1,13,-0.442364,0.000000,-6.196608,20.878865,-0.247764,-4.383362,-2.138687,3.536391,0,12.249851,3.205409,-3349.703387,-6.598124,10.207801,-4.385901,-4.373091,-4.599214,-2.519637,-4.941835,-2.067660,-4.852007,23.913704,4.496061,-4.875950,-1.289758
3,4,13,64,1,11,2.808632,8.139376,0.000000,8.310387,7.416493,5.081580,-1.747566,-2.421811,0,7.378877,3.435874,-6496.208070,-6.944528,2.092179,-3.910026,-1.361372,-3.695701,6.998663,-5.168505,-3.121813,-5.169147,18.443711,7.478396,-4.888263,-1.473757
4,5,20,44,1,7,2.438869,-1.812635,-1.380788,16.874013,2.483339,5.143458,-2.062043,-8.830805,0,11.060563,4.001306,-3423.272870,-6.506065,-5.512043,-4.137751,-0.218374,-4.148412,2.112984,-7.314356,-4.089374,-4.995710,16.072483,-0.310394,-4.664627,-1.439556
8,9,21,22,1,13,-1.700066,3.579329,0.000000,19.004532,0.112770,-5.596050,-0.647825,6.264025,0,11.576480,3.169434,-8866.605450,-6.803791,8.415291,-2.430885,-1.860502,-4.149550,5.025847,-6.484403,-0.028655,-4.741468,37.537688,0.000000,-5.219220,-1.301796
14,15,23,70,1,15,-1.440328,1.471178,-7.433248,22.171605,1.380973,-1.149515,-2.034059,-1.816030,0,8.179702,3.457891,-8949.032821,-6.965592,-4.410356,-4.360464,1.860752,-3.831445,0.000000,-6.169376,-3.899218,-4.892053,40.799830,5.890411,-4.442679,-1.395959


In [33]:
if group_var in clac_mdd_features.columns:
    print(clac_mdd_features.groupby(group_var).sample(n=1, random_state=30)["gender"].value_counts())
else:
    print(clac_mdd_features["gender"].value_counts())

gender
1    506
0    494
Name: count, dtype: int64
gender
1    506
0    494
Name: count, dtype: int64
